In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pydotplus as pp
import seaborn as sns

from IPython.display import display
from IPython.display import Image

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

In [ ]:
class CONFIG:
    INDEXS = ["R2", "MEAN_AE", "MEDIAN_AE", "HIT@100", "DE"]

# Load Dataset

In [ ]:
header = pd.read_csv("../input/facebookdataset/Dataset/header.csv", header=None)
header

In [ ]:
df2 = pd.read_csv("../input/facebookdataset/Dataset/Training/Features_Variant_2.csv",
                               header=None,
                               names=[header.iat[0, i] for i in range(0, 54)])
df3 = pd.read_csv("../input/facebookdataset/Dataset/Training/Features_Variant_3.csv",
                               header=None,
                               names=[header.iat[0, i] for i in range(0, 54)])
df4 = pd.read_csv("../input/facebookdataset/Dataset/Training/Features_Variant_4.csv",
                               header=None,
                               names=[header.iat[0, i] for i in range(0, 54)])
df5 = pd.read_csv("../input/facebookdataset/Dataset/Training/Features_Variant_5.csv",
                               header=None,
                               names=[header.iat[0, i] for i in range(0, 54)])
# training_dataset = pd.concat([df2, df3, df4, df5])
training_dataset = df3
testing_dataset = pd.read_csv("../input/facebookdataset/Dataset/Testing/Features_TestSet.csv",
                             header=None,
                             names=[header.iat[0, i] for i in range(0, 54)])
testing_dataset = testing_dataset.sample(n=1000)
training_dataset

## Feature Engineering

In [ ]:
PCA_start = 4
PCA_end = 34
PCA_dim = 30

In [ ]:
df = testing_dataset
df.insert(46, "posted on weekday", (df["Post published weekday-Monday"] + df["Post published weekday-Tuesday"] + df["Post published weekday-Wednesday"] + df["Post published weekday-Thursday"] + df["Post published weekday-Friday"]).astype("int"))
df.insert(46, "posted on weekend", (df["Post published weekday-Sunday"] + df["Post published weekday-Saturday"]).astype("int"))
df.drop(["Post published weekday-Saturday", "Post published weekday-Sunday", "Post published weekday-Monday", "Post published weekday-Tuesday", "Post published weekday-Wednesday", "Post published weekday-Thursday", "Post published weekday-Friday"], axis = 1, inplace = True)
df.head()
y_test = df.iloc[:, -1:]
X_test = df.iloc[:, :-1]

In [ ]:
# # Separating out the derived value
# derived = df.iloc[:, PCA_start:PCA_end].values
# # Standardizing the features
# norm_derived = StandardScaler().fit_transform(derived)
# # pca = PCA(n_components=10)
# # principal_components = pca.fit_transform(norm_derived)
# # principal_df = pd.DataFrame(data = principal_components
# #              , columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
# pca = PCA(n_components=PCA_dim)
# principal_components = pca.fit_transform(norm_derived)
# principal_df = pd.DataFrame(data = principal_components)
# X_test = pd.concat([df.iloc[:, :PCA_start].reset_index(drop=True), principal_df.reset_index(drop=True), df.iloc[:, PCA_end:-1].reset_index(drop=True)], axis=1)
# y_test = df.iloc[:, -1:]
# X_test

In [ ]:
df = training_dataset
df.insert(46, "posted on weekday", (df["Post published weekday-Monday"] + df["Post published weekday-Tuesday"] + df["Post published weekday-Wednesday"] + df["Post published weekday-Thursday"] + df["Post published weekday-Friday"]).astype("int"))
df.insert(46, "posted on weekend", (df["Post published weekday-Sunday"] + df["Post published weekday-Saturday"]).astype("int"))
df.drop(["Post published weekday-Saturday", "Post published weekday-Sunday", "Post published weekday-Monday", "Post published weekday-Tuesday", "Post published weekday-Wednesday", "Post published weekday-Thursday", "Post published weekday-Friday"], axis = 1, inplace = True)
df.head()
features = df.iloc[:, :-1]
labels = df.iloc[:, -1:]
X_train, X_validate, y_train, y_validate = train_test_split(features, labels, test_size=.3, random_state=42)

In [ ]:
# # Separating out the derived value
# derived = df.iloc[:, PCA_start:PCA_end].values
# # Standardizing the features
# norm_derived = StandardScaler().fit_transform(derived)
# # pca = PCA(n_components=10)
# # principal_components = pca.fit_transform(norm_derived)
# # principal_df = pd.DataFrame(data = principal_components
# #              , columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
# pca = PCA(n_components=PCA_dim)
# principal_components = pca.fit_transform(norm_derived)
# principal_df = pd.DataFrame(data = principal_components)
# features = pd.concat([df.iloc[:, :PCA_start].reset_index(drop=True), principal_df.reset_index(drop=True), df.iloc[:, PCA_end:-1].reset_index(drop=True)], axis=1)
# labels = df.iloc[:, -1:]
# X_train, X_validate, y_train, y_validate = train_test_split(features, labels, test_size=.3, random_state=42)
# X_train
# corr = X_train.corr()
# plt.subplots(figsize=(20, 20))
# sns.heatmap(data=corr, cmap="RdBu_r")
# plt.show()

In [ ]:
train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
train

## Data Sampling

In [ ]:
# '''Simple Data Sample'''
# features = X_train
# labels = y_train

# balance_point = 200

# idx_low = np.where(labels < balance_point)[0]
# idx_high = np.where(labels >= balance_point)[0]

# print("X_train:", X_train.shape)
# print("Before Sample")
# print("#low_target_variable:", idx_low.shape)
# print("#high_target_variable:", idx_high.shape)

# # # # unsample
# # idx_low = np.random.choice(idx_low, size=idx_low.shape[0] // 5, replace=False)
# # oversample
# idx_high = np.random.choice(idx_high, size=idx_high.shape[0] * 10, replace=True)

# # # middle sample
# # size = (idx_high.shape[0] + idx_low.shape[0]) // 4
# # idx_low = np.random.choice(idx_low, size=size, replace=False)
# # idx_high = np.random.choice(idx_high, size=size, replace=True)

# print("After Sample")
# print("#low_target_variable:", idx_low.shape)
# print("#high_target_variable:", idx_high.shape)

# features_low = features.take(idx_low, axis=0)
# features_high = features.take(idx_high, axis=0)
# labels_low = labels.take(idx_low, axis=0)
# labels_high = labels.take(idx_high, axis=0)

# features = pd.concat([features_low, features_high])
# labels = pd.concat([labels_low, labels_high])

# # print(features.shape)
# # print(labels.shape)
# X_train = features
# y_train = labels
# y_train.hist(bins=20, figsize=(20, 20))
# plt.show()

In [ ]:
'''Smogn'''
# !pip install smogn
# import smogn

# smogn_train = smogn.smoter(

#     data = train, 
#     y = "Target Variable"
# )
# smogn_train

In [ ]:
# '''imblearn.SMOTE'''
# from imblearn.over_sampling import SMOTE
# oversample = SMOTE(k_neighbors=2)
# y_train_skewed = y_train // 200
# X_train, y_train = oversample.fit_resample(X_train, y_train_skewed)
# y_train = y_train * 200
y_train.hist(bins=20, figsize=(20, 20))
plt.show()

# Random Forest

## Model Training

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train.values.ravel())
    predict = model.predict(X_test)
    polt_trees_mean_ae(model, X_test, y_test)
    return predict

In [ ]:
def polt_trees_mean_ae(model, X_test, y_test):
    mean_aes = pd.DataFrame(
        {
            "Mean Absolute Error": [mean_absolute_error(tree.predict(X_test), y_test) for tree in model.estimators_], 
            "Single Decision Tree": [i for i in range(0, len(model.estimators_))]
        }
    )
    sns.barplot(x="Single Decision Tree", y="Mean Absolute Error", data=mean_aes)
    plt.show()
    dts_result = pd.DataFrame(
        [model.estimators_[dt].predict(X_test).tolist() for dt in range(len(model.estimators_))]
    )
    print(dts_result.sample(n=10, axis=1).describe())
    sns.boxplot(data=dts_result.sample(n=10, axis=1))
    plt.show()

In [ ]:
def plot_test_predict(y_train, y_test, predict, axes=None, k=None):
    test_predict = pd.concat([pd.DataFrame(y_test).reset_index(drop=True), pd.DataFrame(predict.transpose())], axis=1)
    headers = ["Test", "Predict"]
    test_predict.columns = headers
    if axes is not None and k is not None:
        sns.lineplot(data=test_predict.sample(n=200), ax=axes[k]).set_title("Test Predict Curve k = " + str(k))
    else:
        sns.lineplot(data=test_predict.sample(n=200)).set_title("Test Predict Curve")
    plt.show()
    sns.residplot(x=y_test.to_numpy(), y=predict, lowess=True).set_title("Residuals")
    plt.show()

## Model Evaluation

In [ ]:
def evaluate(model, y_test, predict):
    
    # R-Squared
    r2 = r2_score(y_test, predict)
    
    # Mean Absolute Error
    mean_ae = mean_absolute_error(y_test, predict)
    
    # Median Absolute Error
    median_ae = median_absolute_error(y_test, predict)
    
    y_test_np = y_test.to_numpy().flatten()
    # Defined Error
    de = np.mean(y_test_np * np.abs(y_test_np - predict))
    
    # Hit@100
    y_test_np = y_test.to_numpy().flatten()
    ordered_y_test = np.argsort(y_test_np)
    max_y_test = ordered_y_test[-100:]
    ordered_predict = np.argsort(predict)
    max_predict = ordered_predict[-100:]
    acc = 0
    for pred in max_predict:
        if pred in max_y_test:
            acc += 1
    acc = acc / 100
    
    return r2, mean_ae,median_ae, acc, de

## Final Model

In [ ]:
def train_final_model(X_train, y_train, X_test, y_test, max_depth=None, min_samples_split=2, n_estimators=100):
    # DecisionTreeRegressor
    
    model = RandomForestRegressor(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        n_estimators=n_estimators,
        random_state=42,
    )

    # Model Training
    predict = train_model(model, X_train, y_train, X_test, y_test)
    plot_test_predict(y_train, y_test, predict)

    # Model Evaluation
    r2, mean_ae,median_ae, acc, de = evaluate(model, y_test, predict)

    results = pd.DataFrame({
        "Index": CONFIG.INDEXS, 
        "Mean": [r2, mean_ae,median_ae, acc, de]
    })
    print(results)
    
    return model
    

## Data Sampling

In [ ]:
model = train_final_model(X_train, y_train, X_validate, y_validate, max_depth=12, min_samples_split=0.000001, n_estimators=30)

In [ ]:
predict = model.predict(X_test)
r2, mean_ae,median_ae, acc, de = evaluate(model, y_test, predict)
results = pd.DataFrame({
    "Index": CONFIG.INDEXS, 
    "Mean": [r2, mean_ae,median_ae, acc, de]
})
print(results)

## Random Forest Performance

#### hyperparameter ----- validation ----- test
##### max_depth=12 | min_sample_split=0.0001 | n_estimator=30 | PCA = None
R2=0.73 | MAE=0.503 | HIT@100=0.52 ----- R2=0.351 | MAE=3.50 | HIT@100=0.37 <br>
##### max_depth=12 | min_sample_split=0.0001 | n_estimator=30 | PCA = None
R2=0.73 | MAE=0.503 | HIT@100=0.52 ----- R2=0.351 | MAE=3.50 | HIT@100=0.37 <br>
##### max_depth=12 | min_sample_split=0.0001 | n_estimator=30 | PCA = [4,29]-10 
R2=0.72 | MAE=0.513 | HIT@100=0.48 ----- R2=0.363 | MAE=3.35 | HIT@100=0.32 <br>
##### max_depth=20 | min_sample_split=0.0001 | n_estimator=30 | PCA = None 
R2=0.73 | MAE=0.469 | HIT@100=0.51 ----- R2=0.349 | MAE=3.83 | HIT@100=0.38 <br>
##### max_depth=20 | min_sample_split=0.0001 | n_estimator=30 | PCA = [4,29]-5 
R2=0.70 | MAE=0.519 | HIT@100=0.47 ----- R2=0.312 | MAE=3.77 | HIT@100=0.37 <br>
##### max_depth=20 | min_sample_split=0.0001 | n_estimator=30 | PCA = [4,29]-10 
R2=0.72 | MAE=0.473 | HIT@100=0.48 ----- R2=0.368 | MAE=3.53 | HIT@100=0.34 <br>
##### max_depth=20 | min_sample_split=0.0001 | n_estimator=30 | PCA = [4,34]-10 
R2=0.69 | MAE=0.534 | HIT@100=0.48 ----- R2=0.306 | MAE=5.12 | HIT@100=0.34 <br>
##### max_depth=20 | min_sample_split=0.0001 | n_estimator=30 | PCA = [4,34]-20 
R2=0.70 | MAE=0.531 | HIT@100=0.43 ----- R2=0.298 | MAE=4.74 | HIT@100=0.35 <br>
##### max_depth=20 | min_sample_split=100    | n_estimator=30 | PCA = [4,29]-10 
R2=0.70 | MAE=0.480 | HIT@100=0.47 ----- R2=0.357 | MAE=3.47 | HIT@100=0.32 <br>